### importing libraries

In [30]:
import re 
from textblob import TextBlob 
import tweepy

### Getting Authentication and making connection

In [31]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler("XQlBQ5hfh0JhGMaC3MMKdaY3w", 
    "TMP42E8e4xdF2Xe5V7Xky93xIhU547gxYTVu0iMNOJ7gtgdRjk")
auth.set_access_token("2291593814-MnasoCaroNqh62ub109LbFCuQNIaVruqz8pRMGF", 
    "iFa0VsV5p9BmmYsPegZJSBCweZkVePl7Jit5iTjS0jfT8")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


### Extarcting 100 tweets each

In [32]:
def get_tweets(query, count = 100): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = api.search(q = query+" -filter:retweets", count = count, lang = 'en') 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = (tweet.text) 
            
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

In [76]:
amazon_data = get_tweets("amazon", count = 100)

In [77]:
ebay_data = get_tweets("ebay", count = 100)

In [78]:
shopify_data = get_tweets("shopify", count = 100)

In [79]:
alibaba_data = get_tweets("alibaba", count = 100)

### Saving into dataframe

In [80]:
import pandas as pd
amazon_df = pd.DataFrame.from_dict(amazon_data)
ebay_df = pd.DataFrame.from_dict(ebay_data)
shopify_df = pd.DataFrame.from_dict(shopify_data)
alibaba_df = pd.DataFrame.from_dict(alibaba_data)

In [81]:
alibaba_df

,text
0,Alibaba\nTencent\nByteDance\nHuawei\n\nShut. T...
1,New research shows that Amazon is still ‘leaps...
2,Alikou (Alibaba &amp; Kougyoku) Cherry blossom...
3,MF0017G Watches Men Luxury Brand MINI FOCUS Q...
4,@AlessioUrban Bullish for Amazon and alibaba
...,...
86,@GCPcloud @alibaba_cloud @AWS @Azure I paid. P...
87,@ImIncorrigible Not legal yet. Will be. I almo...
88,China website Alibaba sells “children adrenoch...
89,Why to #my #Cloud account at diffrent places a...


In [82]:
amazon_list = ["amazon" for i in range(len(amazon_df))]
ebay_list = ["ebay" for i in range(len(ebay_df))]
shopify_list = ["shopify" for i in range(len(shopify_df))]
alibabba_list = ["alibaba" for i in range(len(alibaba_df))]

In [83]:
amazon_df["company"] = amazon_list
ebay_df["company"] = ebay_list
shopify_df["company"] = shopify_list
alibaba_df["company"] = alibabba_list


In [84]:
frames = [amazon_df, ebay_df, shopify_df, alibaba_df]

In [85]:
result = pd.concat(frames)

In [86]:
result

,text,company
0,Also me trying to leave five star reviews on A...,amazon
1,@rising @brosandprose After NYC passed on Amaz...,amazon
2,I dislike Amazon very much but that’s my own p...,amazon
3,China used a tiny chip in a huge hack that inf...,amazon
4,@espn If you need copywriters for niches (AdSe...,amazon
...,...,...
86,@GCPcloud @alibaba_cloud @AWS @Azure I paid. P...,alibaba
87,@ImIncorrigible Not legal yet. Will be. I almo...,alibaba
88,China website Alibaba sells “children adrenoch...,alibaba
89,Why to #my #Cloud account at diffrent places a...,alibaba


In [89]:
result.to_csv("brands_data.csv", index = False, header=True)

### Trying streaming with twitter: Future commits

#### Importing relevant libararies


In [39]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import datetime
import csv

In [40]:
class StdOutListener(StreamListener):
     def on_status(self, status):
        
        if (status.lang == "en") & (status.user.followers_count >= 500):
        # Altering tweet text so that it keeps to one line
            text_for_output = "'" + status.text.replace('\n', ' ') +"'"
            csvw.writerow([status.id, status.user.screen_name, # Using datetime to parse it to just get date
                      status.created_at.strftime('%m/%d/%y'),
                      status.user.followers_count,
                      text_for_output])
        return True
     def on_error(self, status_code):
        if status_code == 420:
        # Returning False in on_error disconnects the stream
            return False

In [41]:
l = StdOutListener()
stream = Stream(auth, l)